In [2]:
import pandas as pd
import numpy as np
from rcmndr import Recommender
import importlib
from scipy.sparse import csr_matrix

C:\Users\sony\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
# import our files
plays = pd.read_csv('F:/IHA/lastfm-2k/user_artists.dat',sep='\t')
artists = pd.read_csv('F:/IHA/lastfm-2k/artists.dat',sep='\t',usecols=['id','name'])


In [4]:
fm  = pd.merge(artists, plays, how = 'inner',left_on='id',
                right_on='artistID')
fm = fm.rename(columns={"weight":"userArtistPlays"})

In [5]:
# creating a new column listen count which says how many times the song was listened base on artist name
from collections import Counter
count1 = fm['name']
count2=Counter(count1)

list_count = fm['name'].map(count2)

fm['listen_count'] = list_count

In [6]:
import rcmndr

In [6]:
importlib.reload(rcmndr)

<module 'rcmndr' from 'C:\\Users\\Kanchan\\rcmndr.py'>

In [7]:
rcmnd = rcmndr.Recommender

In [8]:
recommendedArtist = rcmnd.recommendByUser(fm,10)

In [49]:
fm.head()

,id,name,userID,artistID,userArtistPlays
0,1,MALICE MIZER,34,1,212
1,1,MALICE MIZER,274,1,483
2,1,MALICE MIZER,785,1,76
3,2,Diary of Dreams,135,2,1021
4,2,Diary of Dreams,257,2,152


In [9]:
# convert the dataframe into a pivot table
features = fm.pivot(index='artistID', columns='userID', values='listen_count').fillna(0)

# obtain a sparse matrix
matrix_features = csr_matrix(features.values)

In [10]:
fm_unique_artists = fm.drop_duplicates(subset=['artistID']).reset_index(drop=True)[['artistID', 'name']]
decode_id_song = {
    song: i for i, song in 
    enumerate(list(fm_unique_artists.set_index('artistID').loc[features.index].name))
}

In [11]:
model = Recommender(metric='cosine', algorithm='auto', k=20, data=matrix_features, decode_id_song=decode_id_song)

In [56]:
name = 'Lady Gaga'
new_recommendations = model.make_recommendation(new_song=name, n_recommendations=10)

Starting the recommendation process for Lady Gaga ...
... Done


In [57]:
print(f"The recommendations for {name} are:")
print(f"{new_recommendations}")

The recommendations for Lady Gaga are:
['Avril Lavigne', 'Shakira', 'Black Eyed Peas', 'Madonna', 'Ke$ha', 'Beyoncé', 'Christina Aguilera', 'Katy Perry', 'Rihanna', 'Britney Spears']


In [ ]:
from flask import Flask, render_template, request
  
app = Flask(__name__, template_folder='rcmnd_template') 
   
@app.route('/') 
def hello_world(): 
    return render_template('Recommend.html')


@app.route('/resultByID',methods = ['POST'])
def resultByID():
    data = request.form['ID']
    recommendedArtist = rcmnd.recommendByUser(fm,int(data))
    output = recommendedArtist['name'].to_string(index=False)
    return render_template('Recommended.html', prediction_text= "Top recommended artists are : {}".format(output))

@app.route('/resultByArtist',methods = ['POST'])
def resultByArtist():
    data = request.form['artist']
    output = model.make_recommendation(new_song=data, n_recommendations=10)
    return render_template('Recommended.html', prediction_text= "Top recommended artists are : {}".format(output))
 
if __name__ == '__main__':    
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [21/Aug/2020 15:54:12] "GET / HTTP/1.1" 200 -
C:\Users\sony\Anaconda3\lib\site-packages\sklearn\neighbors\base.py:420: UserWarning: Loky-backed parallel loops cannot be nested below threads, setting n_jobs=1
  n_jobs = effective_n_jobs(self.n_jobs)
127.0.0.1 - - [21/Aug/2020 15:54:28] "POST /resultByArtist HTTP/1.1" 200 -


Starting the recommendation process for Lady Gaga ...
... Done


127.0.0.1 - - [21/Aug/2020 15:54:53] "POST /resultByArtist HTTP/1.1" 200 -


Starting the recommendation process for Shakira ...
... Done
